In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,155 kB]
Get:13 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-26 21:37:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2022-09-26 21:37:37 (1.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Extract the dataset you used in Deliverable 1
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_PC_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [5]:
# Re-Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3ARRMDEGED8RD|          5|            0|          0|   N|                Y|
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
| RUXJRZCT6953M|          1|            2|          2|   N|                N|
| R7EO0UO6BPB71|          1|            0|          0|   N|                Y|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [12]:
from pyspark.sql.functions import round, col
# 1. Filter the data and create a new DataFrame or table to retrieve all the rows 
# where the total_votes count is equal to or greater than 20 to pick reviews
# that are more likely to be helpful and to avoid having division by zero errors later on.
vine_votes_ge20_df = vine_df.filter(col("total_votes") > 19)
vine_votes_ge20_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R35OW0SJPJWAHO|          1|           12|        162|   N|                Y|
| RAP2ETU511BOI|          1|            5|         99|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R1RMOCWMV8F4VH|          1|            5|         21|   N|                Y|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|     

In [15]:
# 2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
#  - If you use the SQL option below, you’ll need to cast your columns as floats using WHERE CAST(helpful_votes AS FLOAT)/CAST(total_votes AS FLOAT) >=0.5.
vine_helpful_ge50pct_df = vine_votes_ge20_df\
                            .withColumn("helpful_pct", round(col("helpful_votes") / col("total_votes"), 2))\
                            .filter(col("helpful_pct") > .4999)
vine_helpful_ge50pct_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_pct|
+--------------+-----------+-------------+-----------+----+-----------------+-----------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|       0.77|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|       0.86|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|       0.63|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|       0.86|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|        1.0|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|       0.78|
|R38OLNY5GR1PW6|          5|           44|         44|   N|                Y|        1.0|
|R2GK7RJNIAREIT|          1|           30|         32|   N|                Y|       0.94|
|R2OWRP6FX

In [16]:
# 3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_paid_eq_Y_df = vine_helpful_ge50pct_df.filter(col("vine") == 'Y')
vine_paid_eq_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_pct|
+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|R3J2DPYUH6453D|          5|          136|        153|   Y|                N|       0.89|
|R25QTFLERWK6SJ|          3|          852|        956|   Y|                N|       0.89|
|R1BG71K9B61LI1|          4|           39|         39|   Y|                N|        1.0|
| R8WGX426U56E0|          5|           19|         21|   Y|                N|        0.9|
| R614TCIDD36LE|          4|          131|        147|   Y|                N|       0.89|
|R28X5K1OZBQSBK|          5|           25|         28|   Y|                N|       0.89|
|R2CUDWKAUWGHIT|          5|           19|         24|   Y|                N|       0.79|
| R886ADSI1HBU5|          5|           88|         97|   Y|                N|       0.91|
|R3TTJTAND

In [17]:
# 4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_paid_eq_N_df = vine_helpful_ge50pct_df.filter(col("vine") == 'N')
vine_paid_eq_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_pct|
+--------------+-----------+-------------+-----------+----+-----------------+-----------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|       0.77|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|       0.86|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|       0.63|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|       0.86|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|        1.0|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|       0.78|
|R38OLNY5GR1PW6|          5|           44|         44|   N|                Y|        1.0|
|R2GK7RJNIAREIT|          1|           30|         32|   N|                Y|       0.94|
|R2OWRP6FX

In [29]:
from pyspark.sql.functions import count, sum, when

cnt_cond = lambda cond: sum(when(cond, 1).otherwise(0))

# 5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
vine_review_totals = vine_helpful_ge50pct_df\
                      .groupBy("vine")\
                      .agg(count("star_rating").alias("total_reviews"), \
                           cnt_cond(col("star_rating") == '5').alias("total_5-star"))\
                      .withColumn("5-star_pct", round(col("total_5-star") / col("total_reviews"), 2))
vine_review_totals.show()

+----+-------------+------------+----------+
|vine|total_reviews|total_5-star|5-star_pct|
+----+-------------+------------+----------+
|   Y|         1775|         783|      0.44|
|   N|        77387|       35949|      0.46|
+----+-------------+------------+----------+

